In [1]:
import os
import glob
from IPython.display import Image as IPImage
import pandas as pd             # Pandas
import numpy as np              # NumPy
import matplotlib.pyplot as plt # Matplotlib
import seaborn as sns           # Seaborn
from PIL import Image           # Pillow

# Keras
from keras.layers import Flatten, Dense, Activation, Dropout
from keras import models, optimizers
from keras.models import Sequential
from keras.constraints import MaxNorm
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
from keras.regularizers import l2
from keras.layers import Conv2D, MaxPooling2D
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications import DenseNet121
from keras.applications.densenet import DenseNet121, preprocess_input

# scikit-learn
from sklearn.model_selection import train_test_split


2025-04-20 21:55:36.656898: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from PIL import Image

class BreakHisKerasGenerator(Sequence):
    def __init__(self, csv_path, batch_size=16, shuffle=True, transform=None):
        self.df = pd.read_csv(csv_path)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.transform = transform
        self.indexes = np.arange(len(self.df))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indexes]
        X, y = self.__data_generation(batch_df)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_df):
        X = []
        y = []
        for _, row in batch_df.iterrows():
            img = Image.open(row['filepath']).convert("RGB")
            if self.transform:
                img = self.transform(img)
            else:
                img = img.resize((150, 150))  # Default resizing
            img_array = np.array(img) / 255.0
            X.append(img_array)
            y.append(row['label'])

        X = np.array(X, dtype=np.float32)
        y = np.array(y, dtype=np.int32)
        return X, y

In [3]:
train_generator = BreakHisKerasGenerator(
    csv_path="../data/augmented_train_dataset.csv", batch_size=16, shuffle=True
)

valid_generator = BreakHisKerasGenerator(
    csv_path="../data/new_test.csv", batch_size=16, shuffle=False
)

In [4]:
# Load DenseNet-121 with pre-trained weights
base_model = DenseNet121(
    # TODO download this
    # weights='/kaggle/input/densenet121-weights/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5',
    weights='imagenet',
    include_top=False, 
    input_shape=(150, 150, 3)
)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create model
model = Sequential()

# Add the pre-trained DenseNet-121 base model
model.add(base_model)

# Flatten the output of the base model
model.add(Flatten())

# Add fully connected layers with dropout for regularization
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))

# Additional layers for classification
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Display the summary of the model architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)        │ (None, 4, 4, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │    16,778,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,456,066 (97.11 MB)

 Trainable params: 18,418,562 (70.26 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [5]:
# Specify the file path for saving the visualization image
model_visualization_path = "densenet/nn_architecture.png"

# Plot the model and save the visualization image
plot_model(model, to_file=model_visualization_path, show_shapes=True, show_layer_names=True)

# Display the visualization image
IPImage(filename=model_visualization_path)

NameError: name 'plot_model' is not defined

In [6]:
# Define the path to save the best model checkpoint
checkpoint_path = "densenet/model.h5"

# Create a ModelCheckpoint callback
# This callback saves the model when validation accuracy improves
checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',  # Monitor validation accuracy
    save_best_only=True,     # Save only the best model
    mode='max',              # Save based on the maximum validation accuracy
    verbose=1                # Display progress information
)  

In [7]:
# Compile the model with the Adam optimizer, categorical crossentropy loss, and accuracy metric
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Categorical crossentropy loss for multi-class classification
    metrics=['accuracy']              # Monitor accuracy during training
)

In [8]:
# Train the model using the fit() method
history = model.fit(
    train_generator,                                   # Training data generator
    # steps_per_epoch=toy_generator.samples // toy_generator.batch_size,  # Number of steps per epoch
    epochs=1,                                         # Number of training epochs
    validation_data=valid_generator,                   # Validation data generator
    # validation_steps=valid_generator.samples // valid_generator.batch_size,  # Number of validation steps
    callbacks=[checkpoint]                             # List of callbacks, including the ModelCheckpoint
)

/rds/general/user/js4124/home/anaconda3/envs/ml_py/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.6974 - loss: 0.9937
Epoch 1: val_accuracy improved from -inf to 0.80445, saving model to densenet/model.h5


364/364 ━━━━━━━━━━━━━━━━━━━━ 4127s 11s/step - accuracy: 0.6975 - loss: 0.9926 - val_accuracy: 0.8045 - val_loss: 0.4536


In [9]:
total_layers = len(base_model.layers)
print(f'Total number of layers in the model: {total_layers}')

Total number of layers in the model: 427


In [10]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_layer
1 zero_padding2d
2 conv1_conv
3 conv1_bn
4 conv1_relu
5 zero_padding2d_1
6 pool1
7 conv2_block1_0_bn
8 conv2_block1_0_relu
9 conv2_block1_1_conv
10 conv2_block1_1_bn
11 conv2_block1_1_relu
12 conv2_block1_2_conv
13 conv2_block1_concat
14 conv2_block2_0_bn
15 conv2_block2_0_relu
16 conv2_block2_1_conv
17 conv2_block2_1_bn
18 conv2_block2_1_relu
19 conv2_block2_2_conv
20 conv2_block2_concat
21 conv2_block3_0_bn
22 conv2_block3_0_relu
23 conv2_block3_1_conv
24 conv2_block3_1_bn
25 conv2_block3_1_relu
26 conv2_block3_2_conv
27 conv2_block3_concat
28 conv2_block4_0_bn
29 conv2_block4_0_relu
30 conv2_block4_1_conv
31 conv2_block4_1_bn
32 conv2_block4_1_relu
33 conv2_block4_2_conv
34 conv2_block4_concat
35 conv2_block5_0_bn
36 conv2_block5_0_relu
37 conv2_block5_1_conv
38 conv2_block5_1_bn
39 conv2_block5_1_relu
40 conv2_block5_2_conv
41 conv2_block5_concat
42 conv2_block6_0_bn
43 conv2_block6_0_relu
44 conv2_block6_1_conv
45 conv2_block6_1_bn
46 conv2_block6_1_relu
47 conv2_blo

Block Freezing 

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ================================
# 1. Parse Block Index
# ================================
n_epochs = 1
# idx = int(os.environ.get('PBS_ARRAY_INDEX', 1))  # or manually set for local testing
idx=6
num_blocks_to_unfreeze = idx - 1  # 0 = only top classifier trainable

# ================================
# 2. Load Data
# ================================
os.chdir("/rds/general/user/js4124/home/ML_BreakHis/scr")

train_df = pd.read_csv('../data/augmented_train_dataset.csv')
test_df = pd.read_csv('../data/new_test.csv')

train_df['label'] = train_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)
train_df['filepath'] = train_df['filepath'].str.replace(r"^\.\./", "../data/", regex=True)

image_size = 224
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=(image_size, image_size),
    batch_size=32,
    class_mode='categorical'
)

test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    y_col='label',
    target_size=(image_size, image_size),
    batch_size=16,
    class_mode='categorical'
)

# ================================
# 3. Define Model
# ================================
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# ================================
# 4. Define Layer Ranges for 7 Blocks
# ================================
block_ranges = {
    0: (0, 6),      # Conv + Pooling
    1: (7, 86),     # Dense Block 1 + Transition 1
    2: (87, 186),   # Dense Block 2 + Transition 2
    3: (187, 346),  # Dense Block 3 + Transition 3
    4: (347, 482),  # Dense Block 4
    5: (483, 483),  # Global Average Pooling
    6: (484, 486),  # Classifier
}

total_blocks = len(block_ranges)
if num_blocks_to_unfreeze > total_blocks:
    print(f"Error: max blocks to unfreeze is {total_blocks}")
    sys.exit(1)

# ================================
# 5. Freeze Layers
# ================================
for layer in model.layers:
    layer.trainable = False

if num_blocks_to_unfreeze == 0:
    print("Training classifier only. All base layers remain frozen.")
else:
    # Unfreeze the last N blocks
    blocks_to_unfreeze = list(range(total_blocks - num_blocks_to_unfreeze, total_blocks))
    for i in blocks_to_unfreeze:
        start, end = block_ranges[i]
        for layer in model.layers[start:end + 1]:
            layer.trainable = True
        print(f"Unfreezing block {i} → layers {start} to {end}")

# ================================
# 6. Compile and Train
# ================================
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(f"densenet121_last_{num_blocks_to_unfreeze}_blocks.h5", save_best_only=True)
]

history = model.fit(
    train_generator,
    epochs=n_epochs,
    validation_data=test_generator,
    callbacks=callbacks
)

# ================================
# 7. Plot Results
# ================================
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig(f"training_densenet121_last_{num_blocks_to_unfreeze}_blocks.pdf")
plt.close()


2025-04-20 18:09:34.466835: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 5820 validated image filenames belonging to 2 classes.
Found 1483 validated image filenames belonging to 2 classes.
Unfreezing block 2 → layers 87 to 186
Unfreezing block 3 → layers 187 to 346
Unfreezing block 4 → layers 347 to 482
Unfreezing block 5 → layers 483 to 483
Unfreezing block 6 → layers 484 to 486


/rds/general/user/js4124/home/anaconda3/envs/ml_py/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  8/182 ━━━━━━━━━━━━━━━━━━━━ 3:12:21 66s/step - accuracy: 0.5271 - loss: 0.8455